In [1]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import math

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [2]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
window = glfw.create_window(720, 720, "Transformções Geométricas", None, None)
glfw.make_context_current(window)

In [3]:
vertex_code = """
        attribute vec2 position;
        uniform mat4 mat_transformation;
        void main(){
            gl_Position = mat_transformation * vec4(position,0.0,1.0);
        }
        """

In [4]:
fragment_code = """
        uniform vec4 color;
        void main(){
            gl_FragColor = color;
        }
        """

In [5]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

In [6]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

In [7]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

In [8]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

In [9]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)

In [10]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

In [11]:
num_vertices = 64 # define a "qualidade" do circulo
pi = 3.14
counter = 0
radius = 0.2
vertices = np.zeros(num_vertices, [("position", np.float32, 2)])

angle = 0.0
for counter in range(num_vertices):
    angle += 2*pi/num_vertices 
    x = math.cos(angle)*radius
    y = math.sin(angle)*radius
    vertices[counter] = [x,y]

In [12]:
# Request a buffer slot from GPU
buffer = glGenBuffers(1)
# Make this buffer the default one
glBindBuffer(GL_ARRAY_BUFFER, buffer)

In [13]:
# Upload data
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer)

In [14]:
# Bind the position attribute
# --------------------------------------
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)


In [15]:
loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)

In [16]:
glVertexAttribPointer(loc, 2, GL_FLOAT, False, stride, offset)

In [17]:
loc_color = glGetUniformLocation(program, "color")
R = 0.862
G = 0.654
B = 0.945

In [18]:
t_x = 0.0
t_y = 0.0
def key_event(window,key,scancode,action,mods):
    global t_x, t_y
    if key == 87: t_y += 0.01 #cima
    if key == 83: t_y -= 0.01 #baixo
    if key == 65: t_x -= 0.01 #esquerda
    if key == 68: t_x += 0.01 #direita
    
    
    
glfw.set_key_callback(window,key_event)

In [19]:
# exemplo para matriz de translacao

def mouse_event(window,button,action,mods):
    print("-------")
    # global s_x, s_y
    # if button == 1: #botão direito
    #     s_y += 0.02
    #     s_x += 0.02
    # if button == 0: #botão esquerdo
    #     s_x -= 0.02
    #     s_y -= 0.02

glfw.set_mouse_button_callback(window,mouse_event)

In [20]:
glfw.show_window(window)

In [21]:
while not glfw.window_should_close(window):

    glfw.poll_events() 

    
    glClear(GL_COLOR_BUFFER_BIT) 
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    
    #Draw Triangle
    mat_translation = np.array([    1.0, 0.0, 0.0, t_x, 
                                    0.0, 1.0, 0.0, t_y, 
                                    0.0, 0.0, 1.0, 0.0, 
                                    0.0, 0.0, 0.0, 1.0], np.float32)

    # mat_scale = np.array([    s_x, 0.0, 0.0, 0.0, 
    #                           0.0, s_y, 0.0, 0.0, 
    #                           0.0, 0.0, 1.0, 0.0, 
    #                           0.0, 0.0, 0.0, 1.0], np.float32)

    # mat_rotation = np.array([ math.cos(theta),(-1)*math.sin(theta),  0.0, 0.0, 
    #                           math.sin(theta), math.cos(theta),      0.0, 0.0, 
    #                           0.0,             0.0,                  1.0, 0.0, 
    #                           0.0,             0.0,                  0.0, 1.0], np.float32)

    

    loc = glGetUniformLocation(program, "mat_transformation")
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_translation)

    
    glDrawArrays(GL_TRIANGLE_FAN, 0, len(vertices))
    glUniform4f(loc_color, R, G, B, 1.0) ### modificando a cor do objeto!

    glfw.swap_buffers(window)